In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from scipy.misc import imresize,imread
import tensorflow as tf
from joblib import Parallel,delayed

%matplotlib inline

import os
from os import listdir
from os.path import isfile, join
import shutil
import stat
import collections
from collections import defaultdict

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

import h5py
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from keras import backend as K
import horovod.keras as hvd
import multiprocess as mp

Using TensorFlow backend.


In [ ]:
print(mp.cpu_count())

In [ ]:
p = mp.Pool(4)

In [ ]:
root_dir = 'food-101/images/'
rows = 17
cols = 6
fig, ax = plt.subplots(rows, cols, frameon=False, figsize=(15, 25))
fig.suptitle('Random Image from Each Food Class', fontsize=20)
sorted_food_dirs = sorted(os.listdir(root_dir))
for i in range(rows):
    for j in range(cols):
        try:
            food_dir = sorted_food_dirs[i*cols + j]
        except:
            break
        all_files = os.listdir(os.path.join(root_dir, food_dir))
        rand_img = np.random.choice(all_files)
        img = plt.imread(os.path.join(root_dir, food_dir, rand_img))
        ax[i][j].imshow(img)
        ec = (0, .6, .1)
        fc = (0, .7, .2)
        ax[i][j].text(0, -20, food_dir, size=10, rotation=0,
                ha="left", va="top", 
                bbox=dict(boxstyle="round", ec=ec, fc=fc))
plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

In [2]:
class_to_ix = {}
ix_to_class = {}
with open('accessories/meta/classes.txt', 'r') as txt:
    classes = [l.strip() for l in txt.readlines()]
    class_to_ix = dict(zip(classes, range(len(classes))))
    ix_to_class = dict(zip(range(len(classes)), classes))
    class_to_ix = {v: k for k, v in ix_to_class.items()}
sorted_class_to_ix = collections.OrderedDict(sorted(class_to_ix.items()))

In [ ]:
# Only split files if haven't already
if not os.path.isdir('./food-101/images/test') and not os.path.isdir('./food-101/images/train'):

    def copytree(src, dst, symlinks = False, ignore = None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    def generate_dir_file_map(path):
        dir_files = defaultdict(list)
        with open(path, 'r') as txt:
            files = [l.strip() for l in txt.readlines()]
            for f in files:
                dir_name, id = f.split('/')
                dir_files[dir_name].append(id + '.jpg')
        return dir_files

    train_dir_files = generate_dir_file_map('accessories/meta/train.txt')
    test_dir_files = generate_dir_file_map('accessories/meta/test.txt')


    def ignore_train(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = train_dir_files[subdir]
        return to_ignore

    def ignore_test(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = test_dir_files[subdir]
        return to_ignore
    
    copytree('food-101/images', 'food-101/test', ignore=ignore_train)
    copytree('food-101/images', 'food-101/train', ignore=ignore_test)
    
else:
    print('Train/Test files already copied into separate folders.')

In [9]:
# Load dataset images and resize to meet minimum width and height pixel size
def load_images(root, min_side=299):
    all_imgs = []
    
    all_classes = []
    resize_count = 0
    for i, subdir in enumerate(listdir(root)):
        imgs = listdir(join(root, subdir))
        images = []
        class_ix = class_to_ix[subdir]
        print(i, class_ix, subdir)
        images = Parallel(n_jobs=-1)(delayed(getImgArrRs)(join(root, subdir, img_name)) for img_name in imgs)
        all_imgs.append(images)
        #all_classes.append(class_ix)
    return np.array(all_imgs), np.array(all_classes)

In [8]:
def getImgArrRs(imagePath):
    img_arr = imread(imagePath)
    min_side = 299
    img_arr_rs = img_arr
    invalid_count = 0
    try:
        w, h, _ = img_arr.shape
        if w < min_side:
            wpercent = (min_side/float(w))
            hsize = int((float(h)*float(wpercent)))
            #print('new dims:', min_side, hsize)
            img_arr_rs = imresize(img_arr, (min_side, hsize))
            #img_arr_rs = np.array()
            resize_count += 1
        elif h < min_side:
            hpercent = (min_side/float(h))
            wsize = int((float(w)*float(hpercent)))
            #print('new dims:', wsize, min_side)
            img_arr_rs = imresize(img_arr, (wsize, min_side))
            resize_count += 1
    except:
        print('Skipping bad image: ', imagePath)
        invalid_count += 1
    return img_arr_rs

In [10]:
%%time
X_test, y_test = load_images('food-101/test', min_side=299)

0 30 deviled_eggs
1 55 hot_dog
2 27 creme_brulee
3 19 chicken_quesadilla
4 44 fried_rice
5 61 lobster_roll_sandwich
6 48 greek_salad
7 65 mussels
8 74 peking_duck
9 45 frozen_yogurt
10 94 strawberry_shortcake
11 16 cheesecake
12 77 pork_chop
13 43 fried_calamari
14 62 macaroni_and_cheese
15 37 filet_mignon
16 82 ravioli
17 40 french_fries
18 9 breakfast_burrito
19 54 hot_and_sour_soup
20 11 caesar_salad
21 2 baklava
22 34 eggs_benedict
23 0 apple_pie
24 39 foie_gras
25 68 onion_rings
26 5 beet_salad
27 25 club_sandwich
28 26 crab_cakes
29 8 bread_pudding
30 57 hummus
31 71 paella
32 47 gnocchi
33 83 red_velvet_cake
34 89 shrimp_and_grits
35 28 croque_madame
36 23 churros
37 67 omelette
38 69 oysters
39 14 carrot_cake
40 93 steak
41 97 takoyaki
42 91 spaghetti_carbonara
43 10 bruschetta
44 60 lobster_bisque
45 99 tuna_tartare
46 76 pizza
47 21 chocolate_cake
48 33 edamame
49 85 samosa
50 98 tiramisu
51 95 sushi
52 24 clam_chowder
53 53 hamburger
54 64 miso_soup
55 88 seaweed_salad
56 63

In [6]:
%%time
X_train, y_train = load_images('food-101/train', min_side=299)

0 30 deviled_eggs


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0980s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1078s.) Setting batch_size=14.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 262 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 416 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0320s.) Setting batch_size=12.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   

1 55 hot_dog


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 364 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:    2.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0189s.) Setting batch_size=20.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


2 27 creme_brulee


...

[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 422 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0192s.) Setting batch_size=20.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


100 46 garlic_bread


[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:    1.8s


CPU times: user 1min 36s, sys: 15.2 s, total: 1min 51s
Wall time: 4min 56s


[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:    3.2s finished
